In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC
import numpy as np
from sklearn.metrics import accuracy_score
import warnings
warnings.filterwarnings('ignore')

In [2]:
df = pd.read_csv('../data/heloc_dataset_v1.csv')

* for 'ExternalRiskEstimate' and 'NumInqLast6Mexcl7days' the coefficients are positive so a input number is better bc it gets you closer to one
* for all columns in between the coef is negative so a small number is better bc it gets you closer to one
* to make it uniform I turned the ranges of 'ExternalRiskEstimate' and 'NumInqLast6Mexcl7days' to be negative and made it so the input value becomes negative if it is positive that way the less than the first value in each range is better. For example, 'ExternalRiskEstimate' produces a range of [74,68] which I turned into [-74,-68]. If a person then inputs a value of 80, it becomes -80 which is less than 74 and thus better.

In [41]:
df=pd.DataFrame(df)

d={}

cols=['ExternalRiskEstimate', 'NumTrades60Ever2DerogPubRec', 'NumTrades90Ever2DerogPubRec', 'MaxDelq2PublicRecLast12M', 'MaxDelqEver', 'NumInqLast6M', 'NumInqLast6Mexcl7days']
for i in cols:
    percentiles = pd.Series(df[i])
    x=percentiles.quantile([.4,.6])
    vals = x.values                            
    if i==cols[0] or i==cols[6]:
         d[i]=[-1*vals[1],-1*vals[0]]
    else:
        d[i]=[vals[0],vals[1]]

                                
val = input("Enter your value: ") 
val=int(val)
param = input("Enter the parameter : ") 
if param=='ExternalRiskEstimate' or param== 'NumInqLast6Mexcl7days':
    if val ==abs(val):
        val=val*-1
    else:
        pass

for i in cols:
    if param==i:
        if val<d[i][0]:
            print ("safe")
        elif val>d[i][1]:
            print ("risky")
        else:
            print("neutral")
    else:
        break
 

Enter your value: 80
Enter the parameter : ExternalRiskEstimate
safe


In [ ]:
def label_y(row):
    if row['RiskPerformance'] == 'Good':
        return 1
    else:
        return 0

In [ ]:
df['y'] = df.apply(lambda row: label_y(row),axis=1)

In [ ]:
del df['RiskPerformance']

In [ ]:
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
    
# The function `init_classifiers` returns a list of classifiers to be trained on the datasets
def init_classifiers():
    return([(SVC(), model_names[0], param_grid_svc), 
            (LogisticRegression(), model_names[1], param_grid_logistic),
            (KNeighborsClassifier(), model_names[2], param_grid_knn),
            (GaussianNB(), model_names[3], param_grid_nb),
            (DecisionTreeClassifier(), model_names[4], param_grid_tree),
            (RandomForestClassifier(), model_names[6], param_grid_rf),
            (AdaBoostClassifier(), model_names[7], param_grid_boost)
           ])

# 'model_names' contains the names  that we will use for the above classifiers
model_names = ['SVM','LR','KNN','NB','Tree','QDA','RF','Boosting']

# the training parameters of each model
param_grid_svc = [{'C':[0.1,1],'kernel':['rbf','linear'], 'max_iter':[-1],'random_state':[1]}]
param_grid_logistic = [{'C':[0.1,1], 'penalty':['l1','l2'],'random_state':[1]}]
param_grid_knn = [{},{'n_neighbors':[1,2,3,4]}]
param_grid_nb = [{}]
param_grid_tree = [{'random_state':[1]},{'criterion':['gini'], 'max_depth':[2,3], 'min_samples_split':[3,5],'random_state':[1]}]
param_grid_rf = [{'random_state':[1]},{'n_estimators':[10,30],'max_features':[0.2, 0.3], 'bootstrap':[True],'random_state':[1]}]
param_grid_boost = [{'random_state':[1]},{'n_estimators':[10,20],'learning_rate':[0.1,1],'random_state':[1]}]

In [ ]:
X = df.drop('y', axis= 1)

Y = df['y']



In [ ]:
models = init_classifiers()

In [ ]:
for model in models:
    
    X_train, X_test, y_train, y_test = train_test_split(X,Y)

    clf = GridSearchCV(model[0],model[2],cv = 2)

    clf.fit(X_train,y_train)

    y_pred = clf.predict(X_test)

    print('Model ', model)
    acc = accuracy_score(y_test,y_pred)
    print('Accuracy ', acc)